In [1]:
from pyspark.sql import SparkSession 


spark = SparkSession.builder.appName("Cars_ETL").config("spark.jars", "/home/nader-muhammad/.local/share/DBeaverData/drivers/maven/maven-central/com.mysql/mysql-connector-j-8.2.0.jar").getOrCreate()


df = spark.read.csv(r'/home/nader-muhammad/Documents/Data Engineering/Big_Projects/Data_warehouse2_CarsData/The Data/car_prices.csv', header=True, inferSchema=True)
df.show(5, truncate=False)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/10 06:23:44 WARN Utils: Your hostname, nader-muhammad-HP-255-G7-Notebook-PC, resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlo1)
26/02/10 06:23:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
26/02/10 06:23:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+----+-----+-------------------+----------+-----+------------+-----------------+-----+---------+--------+-----+--------+--------------------------------------+-----+------------+---------------------------------------+
|year|make |model              |trim      |body |transmission|vin              |state|condition|odometer|color|interior|seller                                |mmr  |sellingprice|saledate                               |
+----+-----+-------------------+----------+-----+------------+-----------------+-----+---------+--------+-----+--------+--------------------------------------+-----+------------+---------------------------------------+
|2015|Kia  |Sorento            |LX        |SUV  |automatic   |5xyktca69fg566472|ca   |5        |16639   |white|black   |kia motors america  inc               |20500|21500       |Tue Dec 16 2014 12:30:00 GMT-0800 (PST)|
|2015|Kia  |Sorento            |LX        |SUV  |automatic   |5xyktca69fg561319|ca   |5        |9393    |white|beige   |kia 

### Staging Layer

In [2]:
# df.write.format("jdbc").option("user", "dbeaver").option("password", "Dbeaver@123").option("url", "jdbc:mysql://localhost:3306/ETL_cars").option("dbtable", "unclean_cars_spark")\
#          .option("driver", "com.mysql.cj.jdbc.Driver").mode("append").save()

### ETL Processing

### Uniqueness

In [3]:
df = df.drop_duplicates()

### Name_standarize(Consistency)

In [4]:
from pyspark.sql.functions import *
from pyspark.sql import functions as f 
from pyspark.sql.functions import regexp_replace, upper,lower,initcap

In [5]:
df.select("make").distinct().show(1000)

+-------------+
|         make|
+-------------+
|   Volkswagen|
|   Oldsmobile|
|      ford tk|
|     Infiniti|
|        buick|
|   land rover|
|        Lexus|
|       Jaguar|
|       Saturn|
|         FIAT|
|     Maserati|
|        Scion|
|  Rolls-Royce|
|         Jeep|
|   Mitsubishi|
|   mitsubishi|
|      pontiac|
|          Kia|
|        lexus|
|    Chevrolet|
|     mercedes|
|       toyota|
|        Volvo|
|      Hyundai|
|           vw|
|         Saab|
|     chrysler|
|        Honda|
|     plymouth|
|         MINI|
|         Audi|
|          Ram|
|     Cadillac|
|      lincoln|
|        Isuzu|
|         audi|
|     Plymouth|
|          dot|
|      Pontiac|
|          Geo|
|   Land Rover|
|          bmw|
|Mercedes-Benz|
|         jeep|
|        dodge|
|   hyundai tk|
|      Mercury|
|       Daewoo|
|      Lincoln|
|     Chrysler|
|        Tesla|
|      hyundai|
|         ford|
|          BMW|
|       Suzuki|
|     mazda tk|
|        Acura|
|     cadillac|
|     dodge tk|
|       

In [6]:
df = df.withColumn('make', initcap('make'))

In [7]:
df = df.withColumn('make', regexp_replace('make', 'Vw', 'Volkswagen'))
df = df.withColumn('make', regexp_replace('make', 'Mercedes-b', 'Mercedes-Benz'))
df = df.withColumn('make', regexp_replace('make', 'Mercedes-benz', 'Mercedes-Benz'))
df = df.withColumn('make', regexp_replace('make', 'Rolls-royce', 'Rolls-Royce'))
df = df.withColumn('make', regexp_replace('make', 'Bmw', 'BMW')).withColumn('make', regexp_replace('make', 'Geo', 'GEO'))
df = df.withColumn('make', regexp_replace('make', 'Landrover', 'Range Rover')).withColumn('make', regexp_replace('make', 'Land Rover', 'Range Rover'))


In [8]:
df.select("make").distinct().show(1000)

+----------------+
|            make|
+----------------+
|      Volkswagen|
|      Oldsmobile|
|        Infiniti|
|           Lexus|
|          Jaguar|
|          Saturn|
|        Dodge Tk|
|        Maserati|
|           Scion|
|     Rolls-Royce|
|             Dot|
|            Jeep|
|      Mitsubishi|
|             Kia|
|            Mini|
|       Chevrolet|
|           Volvo|
|         Hyundai|
|            Saab|
|           Honda|
|            Audi|
|             Ram|
|        Cadillac|
|         Ford Tk|
|           Isuzu|
|        Mercedes|
|        Plymouth|
|         Pontiac|
|   Mercedes-Benz|
|     Range Rover|
|      Ford Truck|
|         Mercury|
|          Daewoo|
|         Lincoln|
|        Chrysler|
|           Tesla|
|             BMW|
|          Suzuki|
|           Acura|
|        Mazda Tk|
|             GEO|
|           Smart|
|         Ferrari|
|          Hummer|
|           Buick|
|         Porsche|
|      Chev Truck|
|         Bentley|
|          Toyota|
|    Aston M

In [9]:
df.select("model").distinct().orderBy('model').count()

974

In [10]:
df = df.withColumn('model', initcap('model'))

In [11]:
df.select('model').distinct().count()

852

In [12]:
df.select('trim').distinct().count()

1964

In [13]:
df = df.withColumn('trim', initcap('trim'))

In [14]:
df.select('trim').distinct().count()

1889

In [15]:
df.select('body').distinct().count()

88

In [16]:
df.select('body').distinct().show(1000)

+--------------------+
|                body|
+--------------------+
|       G Convertible|
|                Koup|
|            Quad Cab|
|             G Sedan|
|          Access Cab|
|        Extended Cab|
|           Hatchback|
|            Club Cab|
|             G Coupe|
|           Q60 Coupe|
|         Convertible|
|         CrewMax Cab|
|            King Cab|
|        E-Series Van|
|            Crew Cab|
|          Double Cab|
|          Cab Plus 4|
|         Regular Cab|
|     TSX Sport Wagon|
|            SuperCab|
|                 SUV|
|         CTS-V Coupe|
|               Sedan|
|             Xtracab|
|         convertible|
|           CTS Wagon|
|           SuperCrew|
|       Genesis Coupe|
|               Wagon|
|       Elantra Coupe|
|            Cab Plus|
|                 Van|
|               sedan|
|           CTS Coupe|
|  Beetle Convertible|
|            Mega Cab|
|     G37 Convertible|
| Promaster Cargo Van|
|               Coupe|
|             Minivan|
|     Q60 C

In [17]:
df = df.withColumn('body', initcap('body'))

df = df.withColumn('body', regexp_replace('body','Suv','SUV')).withColumn('body', regexp_replace('body', 'Tsx Sport Wagon', 'TSX Sport Wagon'))

In [18]:
df.select('body').distinct().count()

47

In [19]:
df.select('body').distinct().show(100)

+--------------------+
|                body|
+--------------------+
|       G Convertible|
|                Koup|
|            Quad Cab|
|             G Sedan|
|          Access Cab|
|        Extended Cab|
|           Hatchback|
|            Club Cab|
|             G Coupe|
|           Q60 Coupe|
|        E-series Van|
|         Convertible|
|            King Cab|
|            Crew Cab|
|          Double Cab|
|          Cab Plus 4|
|         Regular Cab|
|     TSX Sport Wagon|
|                 SUV|
|               Sedan|
|             Xtracab|
|           Cts Wagon|
|           Cts Coupe|
|            Supercab|
|       Genesis Coupe|
|               Wagon|
|           Supercrew|
|       Elantra Coupe|
|            Cab Plus|
|                 Van|
|  Beetle Convertible|
|            Mega Cab|
|         Crewmax Cab|
|     G37 Convertible|
| Promaster Cargo Van|
|               Coupe|
|             Minivan|
|         Cts-v Coupe|
|     Q60 Convertible|
|           G37 Coupe|
|          

In [20]:
df.select('transmission').distinct().show(10)

+------------+
|transmission|
+------------+
|   automatic|
|      manual|
|       Sedan|
|       sedan|
|        NULL|
+------------+



In [21]:
df = df.withColumn('transmission', initcap('transmission'))

In [22]:
df.select('transmission').distinct().show(5)

+------------+
|transmission|
+------------+
|   Automatic|
|      Manual|
|       Sedan|
|        NULL|
+------------+



In [23]:
from pyspark.sql.functions import col, substring, concat, lit

df = df.withColumn("vin", concat(
                                 substring(col("vin"), 1, 3), lit("-"),
                                 substring(col("vin"), 4, 3), lit("-"),
                                 substring(col("vin"), 7, 4), lit("-"),
                                 substring(col("vin"), 11, 999)))

In [24]:
df.select('state').distinct().show(100)

+-----------------+
|            state|
+-----------------+
|               pr|
|               ne|
|               il|
|               tx|
|               nv|
|               oh|
|               in|
|               ms|
|               wi|
|               nm|
|               nj|
|               sc|
|               co|
|               on|
|               ab|
|               ok|
|               qc|
|               va|
|               fl|
|               mn|
|               ca|
|               tn|
|               ga|
|               ma|
|               nc|
|               pa|
|               wa|
|               hi|
|               mi|
|               az|
|               ut|
|               md|
|               la|
|               or|
|               ny|
|               mo|
|               al|
|               ns|
|3vwd17aj6fm218641|
|3vwd17aj2fm258506|
|3vwd17aj7fm223475|
|3vwd17aj3fm276741|
|3vwd17aj4fm201708|
|3vwd17aj2fm285365|
|3vwd17aj0fm227318|
|3vwd17aj5fm221322|
|3vwd17aj5fm219943|


In [25]:
df.select('color').distinct().count()

47

In [26]:
df.select('color').distinct().show(1000)

+---------+
|    color|
+---------+
|        —|
| charcoal|
|turquoise|
| burgundy|
|   orange|
|    green|
|   yellow|
|   silver|
|   purple|
|    white|
|     gray|
|     pink|
|     lime|
|      red|
|     gold|
|off-white|
|    beige|
|    black|
|    brown|
|     blue|
|    14872|
|     4802|
|    12655|
|     1167|
|     9410|
|     2172|
|    15719|
|     6388|
|    18384|
|     6864|
|     2711|
|     9887|
|     9837|
|    11034|
|    20627|
|    20379|
|    16633|
|      339|
|     2846|
|     5001|
|     9562|
|     2817|
|    18561|
|     6158|
|      721|
|     5705|
|     NULL|
+---------+



In [27]:
import numpy as np

In [28]:
df = df.withColumn('color', initcap('color'))

df = df.withColumn('color', regexp_replace('color','—', np.nan ))

In [29]:
df.select('color').distinct().count()

47

In [30]:
df.select('color').distinct().show(50)

+---------+
|    color|
+---------+
|   Orange|
|    Beige|
|Turquoise|
| Burgundy|
|   Silver|
|    Green|
|   Purple|
|     Gray|
|     Blue|
|      NaN|
|    White|
|    Brown|
|     Gold|
| Charcoal|
|     Lime|
|    Black|
|   Yellow|
|      Red|
|     Pink|
|Off-white|
|    14872|
|     4802|
|    12655|
|     1167|
|     9410|
|     2172|
|    15719|
|     6388|
|    18384|
|     6864|
|     2711|
|     9887|
|     9837|
|    11034|
|    20627|
|    20379|
|    16633|
|      339|
|     2846|
|     5001|
|     9562|
|     2817|
|    18561|
|     6158|
|      721|
|     5705|
|     NULL|
+---------+



In [31]:
df.select('interior').distinct().show(20)
df.select('interior').distinct().count()


+---------+
| interior|
+---------+
|        —|
| burgundy|
|   orange|
|    green|
|   yellow|
|   silver|
|   purple|
|    white|
|     gray|
|      red|
|off-white|
|     gold|
|    beige|
|      tan|
|    black|
|    brown|
|     blue|
|     NULL|
+---------+



18

In [32]:
df = df.withColumn('interior', initcap('interior'))
df = df.withColumn('interior', regexp_replace('interior', '—', np.nan))

In [33]:
df.select('interior').distinct().show(20)
df.select('interior').distinct().count()

+---------+
| interior|
+---------+
|   Orange|
|    Beige|
| Burgundy|
|      Tan|
|   Silver|
|    Green|
|   Purple|
|     Gray|
|     Blue|
|      NaN|
|    White|
|    Brown|
|     Gold|
|    Black|
|   Yellow|
|      Red|
|Off-white|
|     NULL|
+---------+



18

In [34]:
df.select('seller').distinct().show(10000000)
df.select('seller').distinct().count()

+--------------------+
|              seller|
+--------------------+
|balboa thrift & l...|
|california auto w...|
|repo remarketing/...|
|jaguar land rover...|
|   low gos used cars|
|montclair auto sl...|
|     pa distributors|
|autolenders liqui...|
|   bailey auto plaza|
|southern auto fin...|
|      rock chevrolet|
|premier toyota of...|
|        richmond bmw|
|jaguar land rover...|
|   loeber motors inc|
|    frank kent honda|
|autonation honda ...|
|princeton chevrol...|
|anthony underwood...|
|grossinger toyota...|
|select remarketin...|
|hyundai of everet...|
|reyes auto sales llc|
|onemain rem/ulric...|
|   nolley auto sales|
|central florida p...|
|                mecu|
|jaguar land rover...|
|onemain rem/casey...|
|          rab motors|
|  lester glenn buick|
|laurel bmw of wes...|
|  kia of clarksville|
|ars/budget rac of...|
|san juan motorspo...|
|southern auto fin...|
|ashland motorcars...|
|o'brien's automot...|
|      bmw of el paso|
|courtesy chrysler...|
| sonju fle

14263

In [35]:
df = df.withColumn('seller', initcap('seller'))

In [36]:
df.select('seller').distinct().show(10000000, truncate=False)

+--------------------------------------------------+
|seller                                            |
+--------------------------------------------------+
|Lexus Of Thousand Oaks                            |
|Connell Chevrolet                                 |
|Inter Auto Trade Inc                              |
|H & H Japanese Auto Sales Llc                     |
|Enterprise Car Sales/orange                       |
|Tidalwave Finance Corp                            |
|Valmot                                            |
|Nationwide Remarketing                            |
|Rally Cad Chev Pont Gmc Hyundai Buick             |
|Foursight Capital Llc                             |
|Piazza Honda Of Reading                           |
|O'brien Hyundai Of Fort Meyers                    |
|Jimmy Henry Used Cars Llc                         |
|Remarketing By Ge/manheim Nashville               |
|Chapman Nissan Llc                                |
|Nice Cars Auto Inc                           

### Handling Nulls (completeness)

In [37]:
from pyspark.sql.functions import col, count, when

df.select([count(when(col(c).isNull(), c)).alias(c)
           for c in df.columns]).show()

+----+-----+-----+-----+-----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+
|year| make|model| trim| body|transmission|vin|state|condition|odometer|color|interior|seller|mmr|sellingprice|saledate|
+----+-----+-----+-----+-----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+
|   0|10301|10399|10651|13195|       65352|  4|    0|    11820|      94|  749|     749|     0| 38|          12|      12|
+----+-----+-----+-----+-----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+



In [38]:
df = df.na.fill({'make':'No Data', 'model':'No Model', 'transmission':'No info', 
            'color':'No Color', 'interior':'Missing'})

In [39]:
df = df.na.fill({'trim':'No Value', 'body':'No Data', 'vin':'Not Specified','saledate':'No Date'})

In [40]:
mean_c = df.select(f.mean('condition')).collect()[0][0]
mean_o = df.select(f.mean('odometer')).collect()[0][0]
mean_m = df.select(f.mean('mmr')).collect()[0][0]
mean_s  = df.select(f.mean('sellingprice')).collect()[0][0]

df = df.na.fill({'condition':mean_c,'odometer':mean_o, 'mmr':mean_m, 'sellingprice':mean_s })

In [41]:
df.select('body').distinct().show(10000)

+--------------------+
|                body|
+--------------------+
|       G Convertible|
|                Koup|
|            Quad Cab|
|             G Sedan|
|          Access Cab|
|        Extended Cab|
|           Hatchback|
|            Club Cab|
|             G Coupe|
|           Q60 Coupe|
|        E-series Van|
|         Convertible|
|            King Cab|
|            Crew Cab|
|          Double Cab|
|          Cab Plus 4|
|         Regular Cab|
|     TSX Sport Wagon|
|                 SUV|
|               Sedan|
|             Xtracab|
|           Cts Wagon|
|           Cts Coupe|
|             No Data|
|            Supercab|
|       Genesis Coupe|
|               Wagon|
|           Supercrew|
|       Elantra Coupe|
|            Cab Plus|
|                 Van|
|  Beetle Convertible|
|            Mega Cab|
|         Crewmax Cab|
|     G37 Convertible|
| Promaster Cargo Van|
|               Coupe|
|             Minivan|
|         Cts-v Coupe|
|     Q60 Convertible|
|          

In [42]:
df.select([count(when(col(c).isNull(), c)).alias(c)
            for c in df.columns]).show()

+----+----+-----+----+----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+
|year|make|model|trim|body|transmission|vin|state|condition|odometer|color|interior|seller|mmr|sellingprice|saledate|
+----+----+-----+----+----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+
|   0|   0|    0|   0|   0|           0|  0|    0|        0|       0|    0|       0|     0|  0|           0|       0|
+----+----+-----+----+----+------------+---+-----+---------+--------+-----+--------+------+---+------------+--------+



### Convert DataTypes(Validity)

In [43]:
df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- make: string (nullable = false)
 |-- model: string (nullable = false)
 |-- trim: string (nullable = false)
 |-- body: string (nullable = false)
 |-- transmission: string (nullable = false)
 |-- vin: string (nullable = false)
 |-- state: string (nullable = true)
 |-- condition: integer (nullable = true)
 |-- odometer: integer (nullable = true)
 |-- color: string (nullable = false)
 |-- interior: string (nullable = false)
 |-- seller: string (nullable = true)
 |-- mmr: integer (nullable = true)
 |-- sellingprice: integer (nullable = true)
 |-- saledate: string (nullable = false)



In [44]:
 ### substring(col(""), The start , the number of charcters)

In [45]:
df = df.drop(col('saledate'))

In [47]:
df.write.format('jdbc').option('user', 'dbeaver').option('password', 'Dbeaver@123').option('dbtable', 'clean_cars_spark').option('url', 'jdbc:mysql://localhost:3306/ETL_cars')\
         .option("driver","com.mysql.cj.jdbc.Driver").mode('append').save()

26/02/10 06:27:54 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
